### Prepocesamiento y preparación del dataset.

En primer lugar los audios se convierten a formato mp3 para que estén igual a las grabaciones del call center. Después se aplica un filtro de 300 a 3300 Hz porque es el rango de frecuencias en el que trabajan los codecs de audio más utilizados para telefonía IP. Por último, a los audios se les calcula el los features.
Los casos positivos se incrementan agregando ruido y desplazándolos en tiempo.

#### Imports

In [20]:
import os
import glob
import random
import numpy as np
import statistics
import librosa
from sklearn.model_selection import train_test_split
from database import Database
from audio_process import Augmenter, TimeStretcher, PitchShifter, Reverberator, Distorter, feature_extraction
import re
import pandas as pd

#### Armado del dataset completo

- Se eliminan los audios de menos de 1 segundo

In [2]:

def process_dataset(collection_name, directory, name, n_mfcc, n_start, augment=True ):
    db = Database(collection_name)
    f0min = 300
    f0max = 3300
    directory = directory + '*.mp3'
    print(directory)
    files = glob.glob(directory)
    anger = 0
    files = files[n_start :]
    try:
        for n, path in enumerate(files):
            if n % 1000 == 0:
                print(n)

            file_name = os.path.basename(path)
            file_name = file_name.replace("-", "_")
            in_db = db.select_by_id(file_name, like=True)
            if in_db:
                continue
            emotion = file_name.split('_')[2]
            file_name = f'{name}_{file_name}'
            if emotion == '05' or emotion == 'anger.mp3' or emotion =='ANG':
                emotion = 1
                anger = anger + 1
            else:
                emotion = 0
            
            audio, sr = librosa.load_mp3(path)
            if len(list(audio))/sr < 0.5:
                    continue
            features = feature_extraction(audio, sr, f0min, f0max, n_mfcc, unit="Hertz" )
            if features != 'skip':
                 db.post(file_name, emotion, features, augmentation='')
            processors = [Distorter(), TimeStretcher(), PitchShifter(), Reverberator()]
            names = ['clipping', 'time_stretch', 'pitch_shift', 'reverb']
            augmenter = Augmenter(processors)
            augmented_audios = augmenter.augment(audio, sr)
            for audio, aug in zip(augmented_audios, names):
                features = feature_extraction(audio, sr, f0min, f0max, n_mfcc, unit="Hertz" )
                if features != 'skip':
                    db.post(file_name, emotion, features, augmentation=aug)
                else:
                    print(features)
    except Exception as e:
        print(path)
        raise(e)


In [3]:
Meld = "Data/Meld/*/"
Enterface = "Data/Enterface/*/"
Crema = "Data/CREMA-D/"
IEMOCAP = "Data/IEMOCAP/"
datasets= [Meld, Enterface, Crema, IEMOCAP]
names = ['Meld', 'Enterface', 'Crema', 'IEMOCAP']
process_dataset('IEMOCAP', 'IEMOCAP/', 'iemocap', 16, 0 )


IEMOCAP/*.mp3


In [14]:
db = Database('Meld')
db.print_balance()

positives_no_aug: 1074
negatives_no_aug: 8608
postives_aug: 4296
negatives_aug: 34428
Total_positives: 5370
Total_negatives: 43036


In [6]:
# db = Database("Meld")
# rgx = re.compile("Enterface_.._.._05.*")
# result = db.collection.find({"_id": rgx})
# for r in result:
#     id = r["_id"]
#     r["label"] = 1
#     db.collection.update_one({"_id": id}, {"$set": r})
# rgx = re.compile("Enterface.*")
# result = db.collection.find({"_id": rgx, "label": 1})
# print(len(list(result)))

rgx = re.compile("Meld_.*_anger.*")
result = db.collection.find({"_id": rgx})
for r in result:
    id = r["_id"]
    r["label"] = 1
    db.collection.update_one({"_id": id}, {"$set": r})
rgx = re.compile("Meld.*")
result = db.collection.find({"_id": rgx, "label": 1})
print(len(list(result)))

# rgx = re.compile("Crema_.*_ANG.*")
# result = db.collection.find({"_id": rgx})
# for r in result:
#     id = r["_id"]
#     r["label"] = 1
#     db.collection.update_one({"_id": id}, {"$set": r})
# rgx = re.compile("Crema.*")
# result = db.collection.find({"_id": rgx, "label": 1})
# print(len(list(result)))




0


In [ ]:
iemocap_Ses01M_script03_2_F031_ang.mp3_

In [9]:
db = Database('IEMOCAP')
rgx = re.compile("iemocap_.*_ang.*")
result = db.collection.find({"_id": rgx})
for r in result:
    id = r["_id"]
    r["label"] = 1
    db.collection.update_one({"_id": id}, {"$set": r})
rgx = re.compile("IEMOCAP.*")
result = db.collection.find({"_id": rgx, "label": 1})
print(len(list(result)))

0


In [16]:
db.print_balance()

positives_no_aug: 1074
negatives_no_aug: 8608
postives_aug: 4296
negatives_aug: 34428
Total_positives: 5370
Total_negatives: 43036


In [15]:
l = [{field: np.nan} for field in db.feature_names]
r = {"$or": l}
db.collection.delete_many(r)

In [12]:
rgx = re.compile(".*xxx.*")
db.collection.delete_many({"_id": rgx})

In [3]:

db = Database("IEMOCAP")
db2 = Database("IEMOCAP_Norm_MinMax")
db3 = Database("IEMOCAP_Norm_Std")

In [30]:
_,_, x, y, _, _ = db.get_datasets()


In [7]:
from dataset_preprocessor import MinMax_Scaler, Standard_Scaler, Preprocessor
features_names = db.feature_names

In [31]:
minmax = Preprocessor(scaler=MinMax_Scaler())
standard = Preprocessor(scaler=Standard_Scaler())



In [33]:
df2 = db.dataset_full_aug.drop(columns=features_names)

In [36]:
x_minmax = minmax.scale(x,y)
df_minmax = pd.DataFrame(x_minmax, columns=features_names)
df_minmax = pd.concat([df_minmax, df2], axis=1)
db2.insert_dataframe(df_minmax)

In [37]:
x_std = minmax.scale(x,y)
df_std = pd.DataFrame(x_std, columns=features_names)
df_std = pd.concat([df_std, df2], axis=1)
db3.insert_dataframe(df_std)

BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': 'iemocap_Ses03F_impro06_M010_sad.mp3_'}, 'errmsg': 'E11000 duplicate key error collection: Tesis.IEMOCAP_Norm_Std index: _id_ dup key: { _id: "iemocap_Ses03F_impro06_M010_sad.mp3_" }', 'op': {'mfcc0': 0.037354555225482144, 'mfcc1': 0.7772019189121098, 'mfcc2': 0.45184993395628015, 'mfcc3': 0.5436528772482063, 'mfcc4': 0.5854399940629592, 'mfcc5': 0.38339087248328685, 'mfcc6': 0.542885522130489, 'mfcc7': 0.27575079458093676, 'mfcc8': 0.6296203181894152, 'mfcc9': 0.43191857787076793, 'mfcc10': 0.2529374457393436, 'mfcc11': 0.5739083770430544, 'mfcc12': 0.18759146392852843, 'mfcc13': 0.2963378680050647, 'mfcc14': 0.1670185700743878, 'mfcc15': 0.322569974529594, 'mfcc_delta10': 0.6263422555874288, 'mfcc_delta11': 0.5973730327367147, 'mfcc_delta12': 0.4718026681305149, 'mfcc_delta13': 0.5392592242945502, 'mfcc_delta14': 0.42843454224055066, 'mfcc_d